<h1 style="background-color:#C2B4B9;color:white;text-align: center;padding-top: 5px;padding-bottom: 5px;border-radius: 15px 50px;"><strong><centre>Bird Species Classification Using Convolutional Neural Networks </centre></strong></h1>
<img src="../images/logo_part1.jpg">

<h2 style="background-color:#C2B4B9;color:white;text-align: center;padding-top: 5px;padding-bottom: 5px;border-radius: 15px 50px;"><strong><centre>Methodology: Model Architecture </centre></strong></h2>

#### 03.Model Development
##### Convolutional Neural Networks (CNN) for Bird Species Classification

##### Building a baseline ResNet-50 classifier

We are going to use ResNet-50 model for classification of bird species. ResNet (stands for Residual Networks) is a variant of convolutional neural networks that was [proposed](https://arxiv.org/abs/1512.03385, 'He et. al, 2015') as a solution to the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) of large networks by using the skip or residual connections. 
PyTorch provides the ResNet-50 among the other ready-to-use deep learning models on `torchvision.models`, so we'll instantiate the respective class. Given the dataset of 200 bird species, we will set the argument *num_classes* to that number, and also define the device on which to run the model.

#### Import Libraries:

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


#### Load and Preprocess Data:

In [2]:
data_dir = '../dataset/raw/CUB_200_2011/images/'

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1176 images belonging to 25 classes.
Found 290 images belonging to 25 classes.


#### Define Pretrained Model (ResNet50):

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #NÃO DEVIA ESTAR AQUI DEFINIDO O PATH PARA GUARDAR O MODELO? 
base_model.trainable = False  # Freeze the base model

model_ResNet50 = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(25, activation='softmax')
])

model_ResNet50.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


#### Train Model:

In [7]:
history_ResNet50 = model_ResNet50.fit(train_generator, epochs=10, validation_data=validation_generator)
model_ResNet50.save('models/pretrained/resnet50_bird_species_model.h5')

Epoch 1/10


C:\Users\marin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.0636 - loss: 3.5741 - val_accuracy: 0.0241 - val_loss: 3.2213
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.0308 - loss: 3.4044 - val_accuracy: 0.0586 - val_loss: 3.2101
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.0370 - loss: 3.3678 - val_accuracy: 0.0621 - val_loss: 3.2116
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.0643 - loss: 3.2827 - val_accuracy: 0.0310 - val_loss: 3.2070
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.0605 - loss: 3.2694 - val_accuracy: 0.0552 - val_loss: 3.2015
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.0368 - loss: 3.2517 - val_accuracy: 0.0345 - val_loss: 3.1976
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.0406 - loss: 3.2413 - val_accuracy: 0.0655 - val_loss: 3.1939
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.0429 - loss: 3.2250 - val_accuracy: 0.0621 - val_loss: 3.1859
Epo

#### Plot Training History:

In [ ]:
acc = history_ResNet50.history['accuracy']
val_acc = history_ResNet50.history['val_accuracy']
loss = history_ResNet50.history['loss']
val_loss = history_ResNet50.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()


#### Evaluate Model:

In [ ]:
val_loss, val_acc = model_ResNet50.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc:.2f}')
